# H2O.ai

In [18]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.xgboost import H2OXGBoostEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.automl import H2OAutoML

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [32]:
#h2o.cluster().shutdown()

H2O session _sid_a51d closed.


In [33]:
h2o.init(min_mem_size=6)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_181"; Java(TM) SE Runtime Environment (build 1.8.0_181-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.181-b13, mixed mode)
  Starting server from /anaconda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/dj/0hml51sd01jf891s7llg5fww0000gn/T/tmp6kj2_ecn
  JVM stdout: /var/folders/dj/0hml51sd01jf891s7llg5fww0000gn/T/tmp6kj2_ecn/h2o_rafael_started_from_python.out
  JVM stderr: /var/folders/dj/0hml51sd01jf891s7llg5fww0000gn/T/tmp6kj2_ecn/h2o_rafael_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,04 secs
H2O cluster timezone:,Europe/Amsterdam
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.2
H2O cluster version age:,4 months and 5 days !!!
H2O cluster name:,H2O_from_python_rafael_tebsiy
H2O cluster total nodes:,1
H2O cluster free memory:,5.750 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [34]:
df = pd.read_pickle('../../data/Visual_well_being/merged_features.pickle')

In [35]:
df.head()

,user_id,age,user_followed_by,user_follows,user_posted_photos,Person,Plant,Food,Collage,Animal,...,P,E,R,M,A,PERMA,N_EMO,P_EMO,imagecount,private_account
0,1619510,42.0,364.0,814.0,2037.0,0.220913,0.034855,0.013746,0.025037,0.019146,...,3.000000,5.333333,3.666667,5.333333,5.333333,4.4375,3.000000,3.000000,1515.0,public
1,2829661,36.0,198.0,288.0,124.0,0.306452,0.008065,0.056452,0.008065,0.000000,...,7.666667,8.333333,7.333333,7.666667,8.000000,7.8750,5.666667,7.666667,123.0,public
2,3069744,21.0,311.0,240.0,350.0,0.482857,0.005714,0.037143,0.057143,0.017143,...,4.666667,8.000000,6.333333,4.666667,5.000000,5.6875,5.000000,4.666667,350.0,public
3,3988856,37.0,732.0,943.0,610.0,0.191803,0.011475,0.011475,0.011475,0.018033,...,10.000000,9.333333,10.000000,9.333333,7.666667,9.2500,3.333333,10.000000,609.0,public
4,4185084,29.0,24.0,65.0,64.0,0.281250,0.125000,0.000000,0.015625,0.078125,...,6.333333,4.666667,8.333333,5.666667,8.333333,6.6250,4.333333,6.333333,64.0,public


In [36]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 158
Data columns (total 129 columns):
user_id                                       int64
age                                           float64
user_followed_by                              float64
user_follows                                  float64
user_posted_photos                            float64
Person                                        float64
Plant                                         float64
Food                                          float64
Collage                                       float64
Animal                                        float64
Outdoors                                      float64
Pet                                           float64
Book                                          float64
Dog                                           float64
Canine                                        float64
Sky                                           float64
Alcohol                         

In [39]:
def generate_predictions_automl(train, test, exclude_algos, nfolds, seed, max_models, target):
    
    # Identify predictors and response
    x = train.columns
    y = target
    
    print(x)
    
    x.remove(y)

    # For binary classification, response should be a factor
    #train[y] = train[y].asfactor()
    #test[y] = test[y].asfactor()

    # Number of CV folds (to generate level-one data for stacking)
    nfolds = nfolds

    # Run AutoML for 30 seconds
    aml = H2OAutoML(max_runtime_secs = 3600, nfolds=nfolds, seed=seed, exclude_algos=exclude_algos, max_models=max_models)
    aml.train(x = x, y = y,
              training_frame = train)

    # View the AutoML Leaderboard
    lb = aml.leaderboard

    print(lb)
    
    print_importances_best(lb.as_data_frame().model_id[0], target)

    # releases all objects
    
    h2o.remove_all()
    return lb

def print_importances_best(model_id, feature):

    stack_model = h2o.get_model(model_id)
    
    if (model_id.startswith('Stack')):

        metafit = h2o.get_model(stack_model.metalearner()['name'])
        dict_models = metafit.coef_norm()
        print('Stack is composed by:')
        print(dict_models)

        for key, value in dict_models.items():

            if ((value > 0) & (key != 'Intercept')):

                table = 'coefficients_table'
                
                base_learner = h2o.get_model(key)
                print(key)

                if (key.startswith('GBM') | key.startswith('DRF') | key.startswith('XRT')):

                    table = 'variable_importances'
                    df = pd.DataFrame(base_learner._model_json['output'][table].as_data_frame())
                    print(df)
                    
                    
                    
                else:
                    
                    df_coef = base_learner._model_json['output'][table].as_data_frame()
                    df_coef['abs_standardized_coef'] = [abs(x) for x in df_coef['standardized_coefficients']]
                    df = pd.DataFrame(df_coef.sort_values('abs_standardized_coef', ascending=False))
                    print(df)
                    
                df.to_csv('{}-{}.csv'.format(key, feature))
                    
        
    else:
        
        model = h2o.get_model(model_id)
            
        table = 'coefficients_table'

        if (model_id.startswith('GBM') | model_id.startswith('DRF') | model_id.startswith('XRT')):

            table = 'variable_importances'
            df = pd.DataFrame(model._model_json['output'][table].as_data_frame())
            print(df)

        else:

            df_coef = model._model_json['output'][table].as_data_frame()
            df_coef['abs_standardized_coefficients'] = [abs(x) for x in df_coef.standardized_coefficients]
            df = pd.DataFrame(df_coef.sort_values('abs_standardized_coefficients', ascending=False))
            print(df)
            
        df.to_csv('{}-{}.csv'.format(model_id, feature))

In [ ]:
target_features = ['P', 'E', 'R', 'M', 'A', 'PERMA']

for feature in target_features:
    
    print('### Runnning AutoML for Predicting {} ###'.format(feature))
    
    aux_target_features = target_features.copy()
    
    #Remove remaining features as they could leak info into our dataset
    
    aux_target_features.remove(feature)
    
    df_pred = df.drop(aux_target_features, axis=1).reset_index()
    
    df_train = df_pred.sample(frac=0.7)
    
    h2o_train = h2o.H2OFrame(df_train)
    h2o_test = h2o.H2OFrame(df_pred[~np.isin(df_pred.user_id, df_train.user_id)])
    
    exclude_algos = ['DeepLearning']
    
    generate_predictions_automl(h2o_train, h2o_test, exclude_algos, 5, 123, 10, feature)

### Runnning AutoML for Predicting P ###


/anaconda/lib/python3.6/site-packages/h2o/utils/shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
['level_0', 'user_id', 'age', 'user_followed_by', 'user_follows', 'user_posted_photos', 'Person', 'Plant', 'Food', 'Collage', 'Animal', 'Outdoors', 'Pet', 'Book', 'Dog', 'Canine', 'Sky', 'Alcohol', 'Crowd', 'Toy', 'Cat', 'Coast', 'Tree', 'Beach', 'Sport', 'Teddy Bear', 'Sunlight', 'Light', 'Drawing', 'Sea Life', 'TV', 'Dusk', 'Bikini', 'Sunrise', 'Sunset', 'Swimwear', 'Selfie', 'Beard', 'Woman', 'Cocktail', 'Pool', 'Performer', 'Coffee Cup', 'Tattoo', 'Downtown', 'Musical Instrument', 'Festival', 'City', 'Laptop', 'Pizza', 'Cloud', 'Beer Bottle', 'Money', 'Club', 'Airplane', 'Sketch', 'Sandwich', 'Cafeteria', 'Breakfast', 'Child', 'avg_number_of_faces_over_images_with_faces', 'avg_number_of_faces_over_all_images', 'avg_likes', 'avg_comments', 'happy_flt_pct', 'depressed_flt_pct', 'happy_to_depressed_flt_ratio', 'avg_posts_earl

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_grid_0_AutoML_20181021_131439_model_0,0.0623669,0.249734,0.0623669,0.169169,0.0445388
StackedEnsemble_BestOfFamily_0_AutoML_20181021_131439,0.0726278,0.269495,0.0726278,0.185483,0.0498858
StackedEnsemble_AllModels_0_AutoML_20181021_131439,0.0831821,0.288413,0.0831821,0.201074,0.0509634
GBM_grid_0_AutoML_20181021_131439_model_1,0.166658,0.408237,0.166658,0.287941,0.0674659
GBM_grid_0_AutoML_20181021_131439_model_3,0.166658,0.408237,0.166658,0.287941,0.0674659
GBM_grid_0_AutoML_20181021_131439_model_2,0.166658,0.408237,0.166658,0.287941,0.0674659
GBM_grid_0_AutoML_20181021_131439_model_5,0.272992,0.522486,0.272992,0.374044,0.0810663
DRF_0_AutoML_20181021_131439,0.285541,0.53436,0.285541,0.385002,0.0828118
XRT_0_AutoML_20181021_131439,0.310327,0.55707,0.310327,0.402681,0.088887
GLM_grid_0_AutoML_20181021_131439_model_0,0.895526,0.946322,0.895526,0.635474,nan



                                       variable  relative_importance  \
0                                         P_EMO           768.445129   
1                                           HAP            86.014267   
2                                           P_2            57.005489   
3                                           P_3            52.862606   
4                                           P_1            34.736305   
5                                           R_3            11.450481   
6                                      employed             7.767081   
7                                        income             7.087769   
8                                        gender             4.923351   
9                                           M_3             3.519305   
10                                          E_2             3.283410   
11                                          SAD             2.588686   
12                                          M_1             2.5

...


Stack is composed by:
{'Intercept': 6.864197530864199, 'GBM_grid_0_AutoML_20181021_131619_model_2': 0.2803456132684836, 'GBM_grid_0_AutoML_20181021_131619_model_1': 0.2801505115801086, 'GBM_grid_0_AutoML_20181021_131619_model_3': 0.279278387078555, 'GBM_grid_0_AutoML_20181021_131619_model_0': 0.2969908724087612, 'XRT_0_AutoML_20181021_131619': 0.10809571289708278, 'DRF_0_AutoML_20181021_131619': 0.0, 'GBM_grid_0_AutoML_20181021_131619_model_5': 0.2407229368441978, 'GLM_grid_0_AutoML_20181021_131619_model_0': 0.1656161387426658, 'GBM_grid_0_AutoML_20181021_131619_model_6': 0.0, 'GBM_grid_0_AutoML_20181021_131619_model_4': 0.0}
GBM_grid_0_AutoML_20181021_131619_model_2
                  variable  relative_importance  scaled_importance  percentage
0                      M_2           429.259216           1.000000    0.365637
1                      M_3           362.906799           0.845426    0.309119
2                      M_1           152.265366           0.354717    0.129698
3      

[120 rows x 4 columns]
GBM_grid_0_AutoML_20181021_131619_model_3
                  variable  relative_importance  scaled_importance  percentage
0                      M_2           429.259216           1.000000    0.365637
1                      M_3           362.906799           0.845426    0.309119
2                      M_1           152.265366           0.354717    0.129698
3                    P_EMO            59.444035           0.138481    0.050634
4                      E_2            51.392685           0.119724    0.043776
5                      LON            24.541077           0.057171    0.020904
6            avg_posts_day            11.855874           0.027619    0.010099
7                      R_2             9.975935           0.023240    0.008497
8             user_follows             8.120717           0.018918    0.006917
9                      R_1             7.334156           0.017086    0.006247
10                  income             7.317363           0.017046

[120 rows x 4 columns]
XRT_0_AutoML_20181021_131619
                  variable  relative_importance  scaled_importance  percentage
0                      M_2           769.231934           1.000000    0.292758
1                      M_3           629.787659           0.818723    0.239688
2                      E_2           235.239807           0.305811    0.089529
3                    P_EMO           217.618164           0.282903    0.082822
4                      HAP            98.759514           0.128387    0.037586
5                      P_2            96.012207           0.124816    0.036541
6                      M_1            80.149033           0.104194    0.030504
7                      P_3            47.602272           0.061883    0.018117
8                      R_1            36.881252           0.047946    0.014036
9                      R_2            31.833645           0.041384    0.012115
10                     A_2            24.394445           0.031713    0.009284


[120 rows x 4 columns]
GLM_grid_0_AutoML_20181021_131619_model_0
                                                       names  coefficients  \
0                                                  Intercept     27.195293   
124                                                      M_2      0.212870   
135                                                      M_3      0.193805   
122                                                      M_1      0.173141   
125                                                      E_2      0.111095   
43                                                      Book    -17.398059   
134                                                      R_2      0.060428   
120                                                      A_1      0.059798   
73                                                    Tattoo     34.266826   
57                                                     Light    -14.829417   
130                                                      A_3      0.038293   

[145 rows x 4 columns]
### Runnning AutoML for Predicting A ###
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
['level_0', 'user_id', 'age', 'user_followed_by', 'user_follows', 'user_posted_photos', 'Person', 'Plant', 'Food', 'Collage', 'Animal', 'Outdoors', 'Pet', 'Book', 'Dog', 'Canine', 'Sky', 'Alcohol', 'Crowd', 'Toy', 'Cat', 'Coast', 'Tree', 'Beach', 'Sport', 'Teddy Bear', 'Sunlight', 'Light', 'Drawing', 'Sea Life', 'TV', 'Dusk', 'Bikini', 'Sunrise', 'Sunset', 'Swimwear', 'Selfie', 'Beard', 'Woman', 'Cocktail', 'Pool', 'Performer', 'Coffee Cup', 'Tattoo', 'Downtown', 'Musical Instrument', 'Festival', 'City', 'Laptop', 'Pizza', 'Cloud', 'Beer Bottle', 'Money', 'Club', 'Airplane', 'Sketch', 'Sandwich', 'Cafeteria', 'Breakfast', 'Child', 'avg_number_of_faces_over_images_with_faces', 'avg_number_of_faces_over_all_images', 'avg_likes', 'avg_comments', 'happy_flt_pct', 'dep